## Notebooks to make the queries to the api of the models to get the response

In [1]:
import pandas as pd
import requests
from typing import Dict
import time
import os
import json
import re

### Reading the data

In [2]:
data = pd.read_csv("../02-dataset/data/cleaned/all_questions_df_english.csv")
data

,Unnamed: 0,description,alternatives,year,subject,ground_truth,index_question,text_concat_english,text_concat_portuguese
0,0,Questão 4) A Minor Bird I have wished a bird w...,"['A) culpa por não poder cuidar do pássaro.', ...",2020,"Linguagens, códigos e suas tecnologias",D,4,Question 4) Minor Bird I Have Wished a Bird Wo...,Questão 4) A Minor Bird I have wished a bird w...
1,1,"Questão 5) Finally, Aisha finished with her cu...","['A) reforçam um padrão de beleza.', 'B) retra...",2020,"Linguagens, códigos e suas tecnologias",C,5,"Question 5) Finally, Aisha Finished with Her C...","Questão 5) Finally, Aisha finished with her cu..."
2,2,Questão 01) Pablo Pueblo Regresa un hombre en ...,['A) contrapor a individualidade de um sujeito...,2020,"Linguagens, códigos e suas tecnologias",A,6,Question 01) Pablo Pueblo returns a Hombre en ...,Questão 01) Pablo Pueblo Regresa un hombre en ...
3,3,"Questão 4) Oye, Pito, ésta es: la vida bruta d...",['A) rejeição da língua utilizada por seus ant...,2020,"Linguagens, códigos e suas tecnologias",E,9,"Question 4) Oye, Pito, Estas Es: La Life of Un...","Questão 4) Oye, Pito, ésta es: la vida bruta d..."
4,4,Questão 5) Poco después apareció en casa de El...,['A) determinação para conduzir discussões pes...,2020,"Linguagens, códigos e suas tecnologias",E,10,Question 5) Poco Después Apareció en House of ...,Questão 5) Poco después apareció en casa de El...
...,...,...,...,...,...,...,...,...,...
1001,1001,QuESTÃO 167) Em um jogo disputado em uma mesa ...,"['A) Arthur, pois a soma que escolheu é a meno...",2011,Matemática e suas tecnologias,C,172,Question 167) In a game played at a pool table...,QuESTÃO 167) Em um jogo disputado em uma mesa ...
1002,1002,QuESTÃO 168) É possível usar água ou comida pa...,"['A) 20 mL.', 'B) 24 mL.', 'C) 100 mL.', 'D) 1...",2011,Matemática e suas tecnologias,C,173,Question 168) It is possible to use water or f...,QuESTÃO 168) É possível usar água ou comida pa...
1003,1003,"QuESTÃO 171) Nos últimos cinco anos, 32 mil mu...","['A) 4 mil.', 'B) 9 mil.', 'C) 21 mil.', 'D) 3...",2011,Matemática e suas tecnologias,D,176,"Question 171) In the last five years, 32,000 w...","QuESTÃO 171) Nos últimos cinco anos, 32 mil mu..."
1004,1004,QuESTÃO 174) O setor de recursos humanos de um...,"['A) 24.', 'B) 31.', 'C) 32.', 'D) 88.', 'E) 8...",2011,Matemática e suas tecnologias,E,179,Question 174) The human resources sector of a ...,QuESTÃO 174) O setor de recursos humanos de um...


## Making the requests

In [8]:
get_text(data.iloc[34,:],'pt')

Qual é a alternativa correta para a questão que está entre <>? responda só com a letra que representa a alternativa:

<Questão 57) No caso do Departamento de Defesa dos Estados Unidos, a ênfase está posta no traçado de uma estratégia geral de desarticulação, não só dos inimigos reais como dos potenciais, inserida na concepção preventiva que supõe que a mínima dissidência é um sinal de perigo e de guerra futura. Deve-se ter capacidade para responder a uma guerra convencional tanto quanto para enfrentar um inimigo difuso, atentando simultaneamente para todas as áreas geográficas do planeta. Trata-se, sem dúvida, da estratégia com pretensões mais abrangentes que se desenvolveu até agora.
CECEÑA, A. E. Hegemonias e emancipações no século XXI.
Tomando o texto como parâmetro, qual tendência contemporânea impulsiona a formulação de estratégias mais abrangentes por parte do Estado americano?
A) Erradicação dos conflitos em territórios.
B) Propagação de organizações em redes.
C) Eliminação das 

'Qual é a alternativa correta para a questão que está entre <>? responda só com a letra que representa a alternativa:\n\n<Questão 57) No caso do Departamento de Defesa dos Estados Unidos, a ênfase está posta no traçado de uma estratégia geral de desarticulação, não só dos inimigos reais como dos potenciais, inserida na concepção preventiva que supõe que a mínima dissidência é um sinal de perigo e de guerra futura. Deve-se ter capacidade para responder a uma guerra convencional tanto quanto para enfrentar um inimigo difuso, atentando simultaneamente para todas as áreas geográficas do planeta. Trata-se, sem dúvida, da estratégia com pretensões mais abrangentes que se desenvolveu até agora.\nCECEÑA, A. E. Hegemonias e emancipações no século XXI.\nTomando o texto como parâmetro, qual tendência contemporânea impulsiona a formulação de estratégias mais abrangentes por parte do Estado americano?\nA) Erradicação dos conflitos em territórios.\nB) Propagação de organizações em redes.\nC) Elimina

In [3]:
def get_text(line : pd.Series, lang: str, verbose = True):
    """
    """
    if lang == 'pt':
        prompt = \
        f"""Qual é a alternativa correta para a questão que está entre <>? responda só com a letra que representa a alternativa:\n\n<{line['text_concat_portuguese']}>"""
    else:
        prompt = \
        f"""What is the correct alternative to the question between <>? Answer only with the letter that represents the alternative:\n\n<{line['text_concat_english']}>"""

    if verbose:
        print(prompt)
    return prompt

def make_querie(url : str, text : str, temperature: float = None, stop_tokens: str = None):
    """
    """
    params = {}
    params['query'] = text
    if not(temperature is None):
        params['temperature'] = temperature
    if not(stop_tokens is None):
        params['stop_tokens'] = stop_tokens
    start_time = time.time()
    request = requests.get(url, params = params)

    return_infos = {
        'time_to_run': time.time() - start_time,
        'result': eval(request.text)['result']
    }
    
    return return_infos

def make_all_requests(
        df: pd.DataFrame, 
        models: Dict, 
        temperature: float,
        stop_tokens: str,
        path_to_save: str,
        lang: str,
        base_path: str = "http://localhost:8000",
        verbose: bool = True
    ):
    """
    """
    
    try:
        os.mkdir(path_to_save)
    except:
        pass

    for model,url in models.items():
        
        for index,line in df.iterrows():
            name_arq = f"{line['year']}-{line['index_question']}-{model}.json"
            full_path_arq = f"{path_to_save}/{name_arq}"
            
            # if alredy process, not process
            if os.path.exists(full_path_arq):
                continue
            
            # otherwise compute
            url_req = base_path + url
            
            result = make_querie(
                url = url_req,
                text = get_text(line, lang),
                temperature = temperature,
                stop_tokens = stop_tokens
            )
            
            with open(full_path_arq, 'w') as file:
                file.write(json.dumps(result,indent=2,ensure_ascii=False))
            
            if verbose:
                print(f"--> {model} || {index} || {result}")

        print(f"Run the model {model}")
    

In [34]:
models = {
    "llama_7b":"/models/llama/7b",
    "llama_13b":"/models/llama/13b",
    "alpaca_7b": "/models/alpaca/7b",
    "alpaca_13b": "/models/alpaca/13b",
    "koala_7b": "/models/koala/7b",
    "koala_13b": "/models/koala/13b",
    "vicuna_7b": "/models/vicuna/7b",
    "vicuna_13b": "/models/vicuna/13b"
}

### First experiment. Only concatenating the text and seding to the model and collecting the response
- PT BR
- The models seems to not be capable of explain

In [ ]:
PATH_SAVE_DATA = "data/03-prompt-ask"
LANG = 'pt'
BASE_PATH = "http://localhost:8000"

make_all_requests(
    df = data,
    models = models,
    temperature = 0.1,
    stop_tokens = "pergunta:,</s>,\n",
    path_to_save = PATH_SAVE_DATA,
    lang = LANG,
    base_path = BASE_PATH
)

--> llama_7b || 12 || {'time_to_run': 40.04283547401428, 'result': ' A'}
--> llama_7b || 13 || {'time_to_run': 23.071169137954712, 'result': ' D'}
--> llama_7b || 14 || {'time_to_run': 19.330734252929688, 'result': ' B'}
--> llama_7b || 15 || {'time_to_run': 21.584653854370117, 'result': ' D'}
--> llama_7b || 16 || {'time_to_run': 17.47191834449768, 'result': ' D'}
--> llama_7b || 17 || {'time_to_run': 27.486693620681763, 'result': ' C'}
--> llama_7b || 18 || {'time_to_run': 22.96402335166931, 'result': ' D'}
--> llama_7b || 19 || {'time_to_run': 16.098703861236572, 'result': ' D'}
--> llama_7b || 20 || {'time_to_run': 32.58022475242615, 'result': ' E'}
--> llama_7b || 21 || {'time_to_run': 14.755499839782715, 'result': ' C'}
--> llama_7b || 22 || {'time_to_run': 22.69261622428894, 'result': ' D'}
--> llama_7b || 23 || {'time_to_run': 20.92075753211975, 'result': ' A'}
--> llama_7b || 24 || {'time_to_run': 21.187254190444946, 'result': ' E'}
--> llama_7b || 25 || {'time_to_run': 18.890

--> llama_7b || 123 || {'time_to_run': 18.500961542129517, 'result': ' E'}
--> llama_7b || 124 || {'time_to_run': 17.214871883392334, 'result': ' B'}
--> llama_7b || 125 || {'time_to_run': 19.099855184555054, 'result': ' B'}
--> llama_7b || 126 || {'time_to_run': 14.997325420379639, 'result': ' E'}
--> llama_7b || 127 || {'time_to_run': 24.624825477600098, 'result': ' E'}
--> llama_7b || 128 || {'time_to_run': 17.716935634613037, 'result': ' C'}
--> llama_7b || 129 || {'time_to_run': 13.740974426269531, 'result': ' D'}
--> llama_7b || 130 || {'time_to_run': 15.757895946502686, 'result': ' D'}
--> llama_7b || 131 || {'time_to_run': 18.807557582855225, 'result': ' E'}
--> llama_7b || 132 || {'time_to_run': 17.296566486358643, 'result': ' E'}
--> llama_7b || 133 || {'time_to_run': 8.373213768005371, 'result': ' D'}
--> llama_7b || 134 || {'time_to_run': 18.79178738594055, 'result': ' D'}
--> llama_7b || 135 || {'time_to_run': 16.96054697036743, 'result': ' B'}
--> llama_7b || 136 || {'tim

--> llama_7b || 233 || {'time_to_run': 20.051893949508667, 'result': ' C'}
--> llama_7b || 234 || {'time_to_run': 13.22955870628357, 'result': ' D'}
--> llama_7b || 235 || {'time_to_run': 13.750908613204956, 'result': ' D'}
--> llama_7b || 236 || {'time_to_run': 12.012124061584473, 'result': ' D'}
--> llama_7b || 237 || {'time_to_run': 11.067820310592651, 'result': ' D'}
--> llama_7b || 238 || {'time_to_run': 14.075337648391724, 'result': ' B'}
--> llama_7b || 239 || {'time_to_run': 17.409536123275757, 'result': ' D'}
--> llama_7b || 240 || {'time_to_run': 9.254875183105469, 'result': ' D'}
--> llama_7b || 241 || {'time_to_run': 15.898530960083008, 'result': ' D'}
--> llama_7b || 242 || {'time_to_run': 12.110736846923828, 'result': ' C'}
--> llama_7b || 243 || {'time_to_run': 14.318361759185791, 'result': ' D'}
--> llama_7b || 244 || {'time_to_run': 11.786314725875854, 'result': ' D'}
--> llama_7b || 245 || {'time_to_run': 9.447470664978027, 'result': ' D'}
--> llama_7b || 246 || {'tim

--> llama_7b || 343 || {'time_to_run': 12.649576902389526, 'result': ' D'}
--> llama_7b || 344 || {'time_to_run': 11.959393739700317, 'result': ' D'}
--> llama_7b || 345 || {'time_to_run': 15.470326900482178, 'result': ' D'}
--> llama_7b || 346 || {'time_to_run': 16.444970846176147, 'result': ' D'}
--> llama_7b || 347 || {'time_to_run': 15.314984798431396, 'result': ' D'}
--> llama_7b || 348 || {'time_to_run': 20.732063055038452, 'result': ' D'}
--> llama_7b || 349 || {'time_to_run': 8.883846759796143, 'result': ' D'}
--> llama_7b || 350 || {'time_to_run': 16.111281394958496, 'result': ' 3'}
--> llama_7b || 351 || {'time_to_run': 13.122475147247314, 'result': ' D'}
--> llama_7b || 352 || {'time_to_run': 11.17202353477478, 'result': ' B'}
--> llama_7b || 353 || {'time_to_run': 13.470629453659058, 'result': ' C'}
--> llama_7b || 354 || {'time_to_run': 12.581552028656006, 'result': ' D'}
--> llama_7b || 355 || {'time_to_run': 14.366235733032227, 'result': ' D'}
--> llama_7b || 356 || {'ti

--> llama_7b || 453 || {'time_to_run': 20.00669026374817, 'result': ' C'}
--> llama_7b || 454 || {'time_to_run': 15.288265228271484, 'result': ' E'}
--> llama_7b || 455 || {'time_to_run': 22.883689165115356, 'result': ' D'}
--> llama_7b || 456 || {'time_to_run': 14.690521240234375, 'result': ' D'}
--> llama_7b || 457 || {'time_to_run': 18.556307554244995, 'result': ' D'}
--> llama_7b || 458 || {'time_to_run': 20.061269998550415, 'result': ' D'}
--> llama_7b || 459 || {'time_to_run': 14.907286643981934, 'result': ' D'}
--> llama_7b || 460 || {'time_to_run': 18.808943510055542, 'result': ' D'}
--> llama_7b || 461 || {'time_to_run': 22.637657403945923, 'result': ' D'}
--> llama_7b || 462 || {'time_to_run': 15.216353178024292, 'result': ' D'}
--> llama_7b || 463 || {'time_to_run': 22.083883047103882, 'result': ' D'}
--> llama_7b || 464 || {'time_to_run': 13.38297152519226, 'result': ' D'}
--> llama_7b || 465 || {'time_to_run': 12.070823431015015, 'result': ' D'}
--> llama_7b || 466 || {'ti

--> llama_7b || 563 || {'time_to_run': 25.334847927093506, 'result': ' D'}
--> llama_7b || 564 || {'time_to_run': 24.942429780960083, 'result': ' D'}
--> llama_7b || 565 || {'time_to_run': 28.181259155273438, 'result': ' D'}
--> llama_7b || 566 || {'time_to_run': 21.570388555526733, 'result': ' E'}
--> llama_7b || 567 || {'time_to_run': 17.304627418518066, 'result': ' A'}
--> llama_7b || 568 || {'time_to_run': 14.063740015029907, 'result': ' D'}
--> llama_7b || 569 || {'time_to_run': 18.239150524139404, 'result': ' D'}
--> llama_7b || 570 || {'time_to_run': 18.683669805526733, 'result': ' E'}
--> llama_7b || 571 || {'time_to_run': 15.186842203140259, 'result': ' C'}
--> llama_7b || 572 || {'time_to_run': 14.417832851409912, 'result': ' D'}
--> llama_7b || 573 || {'time_to_run': 24.23077630996704, 'result': ' D'}
--> llama_7b || 574 || {'time_to_run': 18.256433963775635, 'result': ' A'}
--> llama_7b || 575 || {'time_to_run': 28.015621185302734, 'result': ' D'}
--> llama_7b || 576 || {'t

--> llama_7b || 673 || {'time_to_run': 13.125393629074097, 'result': ' D'}
--> llama_7b || 674 || {'time_to_run': 22.829909086227417, 'result': ' D'}
--> llama_7b || 675 || {'time_to_run': 31.556119918823242, 'result': ' E'}
--> llama_7b || 676 || {'time_to_run': 23.112168312072754, 'result': ' E'}
--> llama_7b || 677 || {'time_to_run': 37.69171142578125, 'result': ' A'}


### Reading the data processed

In [ ]:
def get_response(
    path_jsons: str,
    dataframe: pd.DataFrame,
    models: Dict
):
    """
    """
    for name, _ in models.items():
        aux = []
        for index,line in dataframe.iterrows():
            name_arq = f"{line['year']}-{line['index_question']}-{name}.json"
            
            with open(f"{path_jsons}/{name_arq}",'r') as file:
                json_response = json.loads(file.read())
            
            aux.append(json_response)
        dataframe[f'{name}_response'] = aux
            
    return dataframe

data_results = get_response(
    path_jsons = PATH_SAVE_DATA,
    dataframe = data.copy(),
    models = models
)

In [ ]:
data_results

## Defining the heuristics to collect the response of the queries

In [ ]:
def first_caracter(text: str):
    """
    """
    text = text.strip()
    if len(text) > 1 and text[0].isupper() and not(text[1].isalnum()) and text[0] in 'ABCDE':
        return text[0]
    if len(text) == 1 and text[0] in 'ABCDE':
        return text[0]

    return None

def identify_alternative_mid_text(text: str):
    """
    """
    regex1 = " [A-E] ?\)? "
    matches1 = re.findall(regex1,text)
    
    regex2 = " [A-E]\)?"
    matches2 = re.findall(regex2,text)
    
    matches = matches1 + matches2
    
    if len(matches) == 1:
        options = "ABCDE"
        for o in options:
            if o in matches[0]:
                return o
    return None


def run_identify_alternative_result(
    dataframe: pd.DataFrame
):
    """
    """
    heuristics = [
        first_caracter,
        identify_alternative_mid_text
    ]
    
    columns = [c for c in dataframe.columns if c.endswith("_response")]
    
    for column in columns:
        results = []
        for index,line in dataframe.iterrows():
            
            result = None
            for heuristic in heuristics:
                out = heuristic(line[column]['result'])
                if not(out is None):
                    result = out
                    break
            
            if not(result is None):
                results.append(result)
            else:
                results.append('-')
        dataframe[column.replace("_response","_prediction")] = results
    return dataframe
    
    

In [ ]:
df_predictions = run_identify_alternative_result(data_results)
df_predictions

### How much of the all predictions where detected with the label

In [ ]:
def compute_percentage_predictions(
    dataframe: pd.DataFrame
):
    """
    """
    
    columns = [c for c in dataframe.columns if c.endswith("_prediction")]
    
    d = dataframe[columns]
    d = d == '-'
    a,b = d.shape
    d = d.sum().sum()
    
    return 1 - (d / (a*b))

c = compute_percentage_predictions(df_predictions)
print(f"Coverage predictions: {c}")

In [ ]:
def verify_results_text(
    dataframe: pd.DataFrame
):
    """
    """
    
    columns = [c for c in dataframe.columns if c.endswith("_prediction")]
    
    for column in columns:
        
        for index,line in dataframe.iterrows():
            
            if line[column] == '-':
                text = line[column.replace("_prediction","_response")]['result']
                print(f"--> {text}")
verify_results_text(df_predictions)

## Computing the accuracy of the models

In [ ]:
def accuracy(ground_truth, prediction):
    """
    """
    
    eq = ground_truth == prediction
    
    return eq.sum() / len(eq)

def compute_acc_by_year(
    df_predictions: pd.DataFrame
):
    """
    """
    years = list(set(df_predictions['year']))
    years.sort()
    
    columns = [c for c in df_predictions.columns if c.endswith('_prediction')]
    
    list_output = []
    
    
    for column in columns:
        model = column.replace("_prediction","")
        aux = [model]
        for year in years:
            
            df_aux = df_predictions[df_predictions['year'] == year]
            
            aux.append(accuracy(df_aux['ground_truth'],df_aux[column]))
        list_output.append(aux)
            
    return pd.DataFrame(list_output,columns=['model']+years)
    
    

In [ ]:
accuracy(df_predictions['ground_truth'],df_predictions['vicuna_13b_prediction'])

## Accuracy by year

In [ ]:
compute_acc_by_year(df_predictions)

## Accuracy by knowlege area

In [ ]:
def compute_acc_by_knowlege_area(
    df_predictions: pd.DataFrame
):
    """
    """
    subjects = list(set(df_predictions['subject']))
    
    columns = [c for c in df_predictions.columns if c.endswith('_prediction')]
    
    list_output = []
    
    
    for column in columns:
        model = column.replace("_prediction","")
        aux = [model]
        for sub in subjects:
            
            df_aux = df_predictions[df_predictions['subject'] == sub]
            
            aux.append(accuracy(df_aux['ground_truth'],df_aux[column]))
        list_output.append(aux)
            
    return pd.DataFrame(list_output,columns=['model']+subjects)

In [ ]:
compute_acc_by_knowlege_area(df_predictions)

## Overall accuracy

In [ ]:
def compute_acc_overall(
    df_predictions: pd.DataFrame
):
    """
    """
    subjects = list(set(df_predictions['subject']))
    
    columns = [c for c in df_predictions.columns if c.endswith('_prediction')]
    
    list_output = []
    
    
    for column in columns:
        list_output.append(accuracy(df_predictions['ground_truth'],df_predictions[column]))
            
    return pd.DataFrame([list_output],columns=[c.replace("_prediction","") for c in columns ])
    
    

In [ ]:
compute_acc_overall(df_predictions)